In [124]:
dataset = 'A2_EDA_Residential.csv'

In [125]:
"""
get the total culmulative gain from all transactions for large and institutional
over the years
"""

'\nget the total culmulative gain from all transactions for large and institutional\nover the years\n'

In [126]:
import polars as pl

In [127]:
def new_df():
    df = pl.read_csv(dataset)
    df = (
        df.with_columns(
            pl.col("date").str.split("-").list.get(0).cast(int).alias("date")
        )
    )
    df = df.with_columns(
    (pl.col("flip_horizon") / 365.0).alias("flip_horizon_years")
    )
    df = df.with_columns(
        (pl.col("price_diff") / pl.col("flip_horizon_years")).alias("yearly_return")
    )
    df = df.filter(~pl.col("price_diff_pch").is_null())
    return df

df = new_df()
df

address,street,proptype,date,price,prsldate,prslpr,mortgage,usecode,usage,style,yearbuilt,totrooms,bathrooms,bedrooms,lotsize,intersf,lasqbin,amenity,zip,lat,lon,year,cash_sale,buyer_llc_ind,buyer_bus_ind,buyer_trst_ind,buyer_bnk_ind,buyer_gse_ind,buyer_gov_ind,seller_llc_ind,seller_llp_ind,seller_bus_ind,seller_trst_ind,seller_bnk_ind,seller_gse_ind,seller_gov_ind,flip_horizon,flip_ind,buy_side_flip,sell_side_flip,price_diff,price_diff_pch,month_horizon,flip_term,buy_horizon,investor_type_purchase_count,investor_type_sale_count,investor_type_purchase_llc,investor_type_sale_llc,investor_type_purchase_building,investor_type_sale_building,avg_annual_value,investor_type_purchase_value,investor_type_sale_value,current_owner,restype_R1F,tot_owned,buyer_purchases,R1F_total,investor_type_purchase,investor_type_sale,inv_to_inv,flip_horizon_years,yearly_return
str,str,str,i64,f64,str,f64,f64,i64,str,str,i64,i64,i64,i64,i64,i64,str,str,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,str,str,str,str,str,str,str,str,str,f64,str,str,i64,i64,i64,i64,i64,str,str,i64,f64,f64
"""48 Commercial Wharf #48""","""Commercial Wharf""","""RCD""",2012,899000.0,"""07/08/11""",830000.0,0.0,102,"""CONDOMINIUM""","""MID-RISE""",1875,5,2,2,1043,1043,"""1001:1100""","""U""",2110,42.362099,-71.050169,2012,1,0,0,0,0,0,0,0,0,0,0,0,0,0,431,1,0,1,69000.0,8.31,"""14""","""Long-term""","""2017-09-11""","""Non-count investor""","""Non-count investor""","""Non-Small LLC""","""Non-Small LLC""","""Non-building investor""","""Non-building investor""",39086.956522,"""Non-value investor""","""Non-value investor""",1,0,1,1,0,"""Non-investor""","""Non-investor""",0,1.180822,58433.87471
"""1 Charles St S #902""","""Charles St S""","""RCD""",2014,807500.0,"""11/06/08""",665000.0,160000.0,102,"""CONDOMINIUM""","""HIGH-RISE""",2004,3,2,1,926,926,"""901:1000""","""U""",2116,42.351787,-71.067367,2014,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2204,0,0,0,142500.0,21.43,"""24+""","""Non-flip""","""2019-11-19""","""Non-count investor""","""Non-count investor""","""Small LLC""","""Non-Small LLC""","""Non-building investor""","""Non-building investor""",35108.695652,"""Non-value investor""","""Non-value investor""",0,0,0,1,0,"""Small""","""Non-investor""",0,6.038356,23599.137931
"""1 Avery St #31B""","""Avery St""","""RCD""",2013,3.475e6,"""08/29/08""",3.05e6,0.0,102,"""CONDOMINIUM""","""HIGH-RISE""",2000,7,5,3,2667,2667,"""2601:2700""","""U""",2111,42.35337,-71.062515,2013,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1826,0,0,0,425000.0,13.93,"""24+""","""Non-flip""","""2018-08-29""","""Non-count investor""","""Non-count investor""","""Non-Small LLC""","""Non-Small LLC""","""Non-building investor""","""Non-building investor""",151086.956522,"""Small""","""Small""",1,0,1,1,0,"""Small""","""Small""",1,5.00274,84953.450164
"""1 Avery St #10D""","""Avery St""","""RCD""",2009,845000.0,"""03/29/07""",845000.0,0.0,102,"""CONDOMINIUM""","""HIGH-RISE""",2000,4,2,2,1087,1087,"""1001:1100""","""U""",2111,42.35337,-71.062515,2009,1,0,0,1,0,0,0,0,0,1,0,0,0,0,null,0,0,0,0.0,0.0,null,"""Non-flip""","""2014-04-30""","""Non-count investor""","""Non-count investor""","""Non-Small LLC""","""Non-Small LLC""","""Non-building investor""","""Non-building investor""",36739.130435,"""Non-value investor""","""Non-value investor""",0,0,0,1,0,"""Non-investor""","""Non-investor""",0,null,null
"""1 Avery St #10B""","""Avery St""","""RCD""",2010,1.085e6,"""08/28/02""",1.125e6,0.0,102,"""CONDOMINIUM""","""HIGH-RISE""",2000,6,3,2,1675,1675,"""1601:1700""","""U""",2111,42.35337,-71.062515,2010,1,0,0,1,0,0,0,0,0,0,0,0,0,0,null,0,0,0,-40000.0,-3.56,null,"""Non-flip""","""2015-12-30""","""Non-count investor""","""Non-count investor""","""Non-Small LLC""","""Non-Small LLC""","""Non-building investor""","""Non-building investor""",47173.913043,"""Non-value investor""","""Non-value investor""",1,0,1,1,0,"""Non-investor""","""Non-investor""",0,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…

In [128]:
# All transactions
# Wow, large and institutional ones make a lot of money
target_set = ["Large", "Institutional"]
rich_investor_sales =  df.filter(pl.col("investor_type_sale").is_in(target_set))
rich_investor_sales = rich_investor_sales.with_columns(
    pl.lit(True).alias("isLarge")
)

rich_investor_sales.group_by(["year"]).agg(
    pl.col("price_diff_pch").median()
).sort("year")

year,price_diff_pch
i64,f64
2000,22.55
2001,33.05
2002,17.03
2003,14.36
2004,14.15
…,…
2018,33.1
2019,32.29
2020,76.92


In [129]:
rich_investor_sales.filter(pl.col("year") == 2020)

address,street,proptype,date,price,prsldate,prslpr,mortgage,usecode,usage,style,yearbuilt,totrooms,bathrooms,bedrooms,lotsize,intersf,lasqbin,amenity,zip,lat,lon,year,cash_sale,buyer_llc_ind,buyer_bus_ind,buyer_trst_ind,buyer_bnk_ind,buyer_gse_ind,buyer_gov_ind,seller_llc_ind,seller_llp_ind,seller_bus_ind,seller_trst_ind,seller_bnk_ind,seller_gse_ind,seller_gov_ind,flip_horizon,flip_ind,buy_side_flip,sell_side_flip,price_diff,price_diff_pch,month_horizon,flip_term,buy_horizon,investor_type_purchase_count,investor_type_sale_count,investor_type_purchase_llc,investor_type_sale_llc,investor_type_purchase_building,investor_type_sale_building,avg_annual_value,investor_type_purchase_value,investor_type_sale_value,current_owner,restype_R1F,tot_owned,buyer_purchases,R1F_total,investor_type_purchase,investor_type_sale,inv_to_inv,flip_horizon_years,yearly_return,isLarge
str,str,str,i64,f64,str,f64,f64,i64,str,str,i64,i64,i64,i64,i64,i64,str,str,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,str,str,str,str,str,str,str,str,str,f64,str,str,i64,i64,i64,i64,i64,str,str,i64,f64,f64,bool
"""245 Summer St""","""Summer St""","""COM""",2020,7.285e8,"""10/14/04""",3.07e7,6.84e8,340,"""GEN OFFICE""",null,1974,0,0,0,79701,910460,"""910401:910500""",null,2210,42.351354,-71.052911,2020,0,1,0,0,0,0,0,0,0,0,0,0,0,0,5655,0,0,0,6.978e8,2272.96,"""24+""","""Non-flip""","""2025-04-08""","""Non-count investor""","""Non-count investor""","""Small LLC""","""Non-Small LLC""","""Non-building investor""","""Non-building investor""",3.1674e7,"""Institutional""","""Institutional""",1,null,1,1,null,"""Institutional""","""Institutional""",1,15.493151,4.5039e7,true
"""242 Beacon St #8""","""Beacon St""","""RCD""",2020,2.2e6,"""04/09/18""",1.95e6,0.0,102,"""CONDOMINIUM""","""ROW-MIDDLE""",1884,6,3,3,1986,1986,"""1901:2000""","""U""",2116,42.353416,-71.079975,2020,1,0,0,0,0,0,0,0,0,0,0,0,0,0,809,0,0,0,250000.0,12.82,"""24+""","""Non-flip""","""2025-06-26""","""Non-count investor""","""Large""","""Non-Small LLC""","""Non-Small LLC""","""Non-building investor""","""Non-building investor""",95652.173913,"""Non-value investor""","""Non-value investor""",0,0,0,1,0,"""Non-investor""","""Large""",0,2.216438,112793.572311,true
"""400 Stuart St #15A""","""Stuart St""","""RCD""",2020,2.195e6,"""06/27/16""",2.25e6,1.756e6,102,"""CONDOMINIUM""","""HIGH-RISE""",2009,4,2,2,1409,1409,"""1401:1500""","""U""",2116,42.348853,-71.074296,2020,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1645,0,0,0,-55000.0,-2.44,"""24+""","""Non-flip""","""2025-12-28""","""Non-count investor""","""Large""","""Non-Small LLC""","""Non-Small LLC""","""Non-building investor""","""Non-building investor""",95434.782609,"""Non-value investor""","""Non-value investor""",1,0,1,1,0,"""Non-investor""","""Large""",0,4.506849,-12203.647416,true
"""175 Beacon St""","""Beacon St""","""R1F""",2020,1.175e7,"""09/06/17""",6.3e6,0.0,101,"""1-FAM RES""","""ROW-MIDDLE""",1905,11,7,4,2016,7109,"""7101:7200""","""U""",2116,42.354449,-71.075838,2020,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1133,0,0,0,5.45e6,86.51,"""24+""","""Non-flip""","""2025-10-13""","""Non-count investor""","""Non-count investor""","""Small LLC""","""Small LLC""","""Non-building investor""","""Non-building investor""",510869.565217,"""Non-value investor""","""Large""",1,1,1,1,1,"""Small""","""Large""",1,3.10411,1.7557e6,true
"""34 Fayette St #2""","""Fayette St""","""RCD""",2020,1.875e6,"""05/17/18""",1.58e6,1.275e6,102,"""CONDOMINIUM""","""ROW-END""",1899,6,4,3,1832,1832,"""1801:1900""","""U""",2116,42.348545,-71.067824,2020,0,0,0,0,0,0,0,0,0,0,0,0,0,0,805,0,0,0,295000.0,18.67,"""24+""","""Non-flip""","""2025-07-30""","""Non-count investor""","""Institutional""","""Non-Small LLC""","""Small LLC""","""Non-building investor""","""Non-building investor""",170697.826087,"""Small""","""Small""",1,0,2,5,2,"""Small""","""Institutional""",1,2.205479,133757.763975,true
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…